In [2]:
#  this code can be used to scrape base data from pages (excluding retail price)
#  36,912 transactions is ~10mb of data when saved down...
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import pandas as pd
from math import ceil
import os

from itertools import cycle
import traceback

def return_response(url):
    headers = {
            'sec-fetch-mode': 'cors',
            'accept-language': 'en-US,en;q=0.9',
            'authorization': '',
            'x-requested-with': 'XMLHttpRequest',
            'appos': 'web',
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36',
            'accept': '*/*',
            'authority': 'stockx.com',
            'sec-fetch-site': 'same-origin',
            'appversion': '0.1',
    'accept': 'application/json'}
    
    return requests.get(url, headers=headers)
    #dir(r)
    #help(r)
            
def play_alert():
    file = '/Users/janzajac/Desktop/Projects/Sneakers/sms-alert-3-daniel_simon.mp3'
    os.system("afplay " + file)

In [3]:
from typing import List, Dict

#  only extracting transaction amount, date and shoe size as important variables
def clean_transactions(transactions: List[Dict]):
    for t in transactions:
        del t['chainId']
        del t['productId']
        del t['state']
        del t['customerId']
        del t['skuUuid']
        del t['localCurrency']
        del t['localAmount']
        
    return transactions
        
#  suggested limit is 10,000
def get_total_transactions(sku):
    url = f'https://stockx.com/api/products/{sku}/activity?state=480&currency=USD&limit=5&page=1&sort=createdAt&order=DESC&country=US'
    resp = return_response(url)
    
    if resp.ok:     
        resp_data = resp.json()
        return resp_data['Pagination']['total']
    
    else:
        play_alert()
        print(f'broke on sku {sku}')
        raise SystemExit

def collect_transaction_data(sku):
    total = get_total_transactions(sku)
    limit = 10000
    pages = ceil(total/limit)
    
    transactions = []
    for page in range(1,pages+1):
        url = f'https://stockx.com/api/products/{sku}/activity?state=480&currency=USD&limit={limit}&page={page}&sort=createdAt&order=DESC&country=US'
        resp = return_response(url)
        if resp.ok:
            resp_data = resp.json()
            transactions.append(resp_data['ProductActivity'])
        
        else:
            play_alert()
            print(f'broke on sku {sku}')
            raise SystemExit
        
    transactions_flat = [item for sublist in transactions for item in sublist]
    transactions_clean = clean_transactions(transactions_flat)
    return transactions_clean

In [11]:
adi_df = pd.read_csv('../Sneakers_backup/persistence/adidas.clean.csv')
df = adi_df

In [10]:
skus = adi_df['sku']

In [12]:
skus = df['sku']
collect_transaction_data(skus[0])

[{'amount': 281, 'createdAt': '2020-03-29T16:11:34+00:00', 'shoeSize': '10.5'},
 {'amount': 298, 'createdAt': '2020-03-29T15:42:25+00:00', 'shoeSize': '12'},
 {'amount': 370, 'createdAt': '2020-03-29T15:30:39+00:00', 'shoeSize': '6.5'},
 {'amount': 340, 'createdAt': '2020-03-29T15:28:18+00:00', 'shoeSize': '6'},
 {'amount': 285.2196,
  'createdAt': '2020-03-29T15:12:53+00:00',
  'shoeSize': '9.5'},
 {'amount': 293.9381,
  'createdAt': '2020-03-29T15:05:10+00:00',
  'shoeSize': '8.5'},
 {'amount': 345, 'createdAt': '2020-03-29T15:03:17+00:00', 'shoeSize': '5.5'},
 {'amount': 264, 'createdAt': '2020-03-29T14:29:49+00:00', 'shoeSize': '9.5'},
 {'amount': 260, 'createdAt': '2020-03-29T14:07:56+00:00', 'shoeSize': '11'},
 {'amount': 365, 'createdAt': '2020-03-29T14:03:33+00:00', 'shoeSize': '6.5'},
 {'amount': 275, 'createdAt': '2020-03-29T13:54:31+00:00', 'shoeSize': '11'},
 {'amount': 291, 'createdAt': '2020-03-29T12:57:37+00:00', 'shoeSize': '8.5'},
 {'amount': 270.2778,
  'createdAt': '

In [13]:
test[-1]

{'amount': 500, 'createdAt': '2019-11-11T02:28:55+00:00', 'shoeSize': '10'}

In [151]:
transactions = {}
wait_seconds = [1, 2, 4, 5, 6, 7]
wait_pool = cycle(wait_seconds)

In [184]:
#  adding retail price to the df
for sku in (skus):
    if sku not in transactions:
        wait = next(wait_pool)
        time.sleep(wait)
        print(sku)
        transactions[sku] = collect_transaction_data(sku)

In [181]:
len(transactions)

949

In [185]:
t_df = pd.DataFrame(list(transactions.items()), columns=['sku', 'transactionHistory'])

In [186]:
len(t_df.iloc[2]['transactionHistory'])

9232

In [187]:
df_merged = pd.merge(adi_df, t_df, on='sku', how='outer')

In [188]:
df_merged.count()

color                 949
model                 949
itemCondition         949
name                  949
brand                 949
releaseDate           949
sku                   949
image                 949
url                   949
retailPrice($)        939
transactionHistory    949
dtype: int64

In [189]:
df_merged['transactionHistory']

0      [{'amount': 320, 'createdAt': '2020-01-18T17:5...
1      [{'amount': 353, 'createdAt': '2020-01-18T18:1...
2      [{'amount': 278, 'createdAt': '2020-01-18T18:2...
3      [{'amount': 548.9947, 'createdAt': '2020-01-18...
4      [{'amount': 300, 'createdAt': '2020-01-18T18:0...
                             ...                        
944    [{'amount': 102, 'createdAt': '2020-01-17T23:5...
945    [{'amount': 114, 'createdAt': '2020-01-17T20:0...
946    [{'amount': 140, 'createdAt': '2020-01-19T21:0...
947    [{'amount': 199, 'createdAt': '2020-01-12T03:0...
948    [{'amount': 122, 'createdAt': '2020-01-17T21:1...
Name: transactionHistory, Length: 949, dtype: object

In [190]:
df_merged.to_csv('/Volumes/Elements/Sneaker_data/adidas.csv', index=False)

In [1]:
adi_df = pd.read_csv('/Volumes/Elements/Sneaker_data/adidas.csv')
adi_df.head()

NameError: name 'pd' is not defined